<a href="https://colab.research.google.com/github/kotaro0918/Gemini_RAG/blob/main/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このプログラムを実行する前に、GeminiのAPIキーを発行してシークレットキーに保管してください
またNDCのドキュメントをドライブに保存してください

NDCのtxtデータはEmbeddingのレポジトリに落ちてます
(余裕があったらRAGのレポジトリにもおいておきますね)

In [3]:
#必要なライブラリのインストール
!pip install langchain
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00


In [4]:
#Google Driveへのアクセス許可
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#必要なライブラリインポート
# Embedding用
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# テキストファイルを読み込む
from langchain.document_loaders import TextLoader

In [6]:
# 環境変数の準備
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
txt_path = "/content/drive/MyDrive/NDCdata/doc_class.txt"

In [8]:
#テキストデータの読み込み
loader = TextLoader(txt_path)
documents = loader.load()

In [34]:
#テキストデータの分割
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [35]:
import pandas as pd
df=pd.DataFrame(data=docs,columns=["doc","source","type"])
display(df)

,doc,source,type
0,"(page_content, 国立国会図書館\n「日本十進分類法（NDC）新訂 10 版」\...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
1,"(page_content, 目次\n本基準の凡例 .......................","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
2,"(page_content, 本基準の凡例\n\n\n各項目には、分類基準に関連する分類記...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
3,"(page_content, \n\n固有補助表を使用しない場合は、個別に明記する。\n\...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
4,"(page_content, 情報産業．情報サービス\n\n個々の企業史・誌も，ここに収める...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
...,...,...,...
215,"(page_content, 団体，企業体．会社誌，研究調査機関，教育・養成機関\n\nこれ...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
216,"(page_content, 分を使用し，地理区分を行わない；ただし，項目名が団体に該当する...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
217,"(page_content, 115\n\n －088 資料集\n特定主題についての史料集に...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"
218,"(page_content, 相関索引・使用法編\n使用法\nI 2.1 4）複数主題\nも...","(metadata, {'source': '/content/drive/MyDrive/...","(type, Document)"


In [36]:
#分割したドキュメントをEmbedding
docs=list(docs)
result = genai.embed_content(
    model="models/embedding-001",
    content=df["doc"],
    task_type="retrieval_document",
    title="Embedding of list of strings")
df["Embedding"]=result["embedding"]

In [37]:
#得られたベクトルを標準化
import numpy as np
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
       return v
    return v / norm

df["Embedding"]=df["Embedding"].map(normalize_vector)
passage_embeddings=df["Embedding"]

In [42]:
#queryをEmbedding
result = genai.embed_content(
    model="models/embedding-001",
    content="""
    手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """,
    task_type="retrieval_document",
    title="Embedding of list of strings")
query_embedding = normalize_vector(result["embedding"])

In [43]:
#得られたベクトルの内積を計算
scores = []
for content in passage_embeddings:
  scores.append(np.dot(content,query_embedding))

In [44]:
#内積を降順に並べ替えた時のインデックスを取得
def get_sorted_indices(lst):
    return [i for i, v in sorted(enumerate(lst), key=lambda x: x[1],reverse = True)]
scores_indices = get_sorted_indices(scores)

In [45]:
#コサイン類似度の高い３つの文章を出力
print(df["doc"][scores_indices[0:3]])

38    (page_content, 17\n\n（2）個々の戦争における戦没者・従軍者の列伝は，...
42    (page_content, 19\n\n（3）土地，風土のことよりも社会事情（政治・経済...
69    (page_content, 35\n\n細目表に掲出されていなくても，独自の学説・体系を...
Name: doc, dtype: object


次に実際にLLMを通して効果を確認してみます

In [1]:
# パッケージのインストール
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [18]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
あなたは誠実で優秀な日本人のアシスタントです。
与えられた文章に対して適切な分類項目を答えてください
この際分類項目に関する情報が与えられます
"""

def q(text):
	prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
	    bos_token=tokenizer.bos_token,
	    b_inst=B_INST,
	    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
	    prompt=text,
	    e_inst=E_INST,
	)
	with torch.no_grad():
	    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
	    output_ids = model.generate(
	        token_ids.to(model.device),
	        max_new_tokens=1024,
	        pad_token_id=tokenizer.pad_token_id,
	        eos_token_id=tokenizer.eos_token_id,
	    )
	output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
	print(output)

In [19]:
q("""手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


与えられた文章に対して適切な分類項目は「鉄道」です。

文章中には「手宮線は、北海道開拓の物資輸送の拠点として発展しました。」、「北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。」といった記述があり、手宮線に関する情報が載っています。


In [48]:
text="""手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """
query = text +str(df["doc"][scores_indices[0]])

In [49]:
print(query)

手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    ('page_content', '17\n\n\x0c（2）個々の戦争における戦没者・従軍者の列伝は，個々の戦争の下に収める；ただし，明\n治・大正・昭和にわたる戦没者・従軍者の列伝は，281 に収める；軍人の列伝は，392.8 に\n収める\n（3）特定の地域の関係者としてまとめられた列伝は，その地域の列伝として扱う；他国に\n居住する一民族に関するものは，母国ではなく居住している国または地域の下に収める\n【列伝と個人伝記】\n一人の中心人物をめぐる列伝は，その中心人物の個人伝記として分類する\n280.2 墓誌．墓銘\n各国・各地域における墓誌・墓銘は，281／287 に収める；280.2 と 281.02 以外の箇所で－02\nは付加しない\n280.38\n\n肖像集')


In [50]:
q(query)

与えられた文章に対して適切な分類項目は「280 列伝」です。

この文章は手宮線に関するもので、各国・各地域における墓誌・墓銘についての情報も含まれています。
